<a href="https://colab.research.google.com/github/laure-m/ProxyFaces_Course/blob/main/proxyFaces_YOLOv8_Det_Seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PROXY FACES UCLA AUD SPRING TECH SEMINAR 
#YOLOv8 Detection + Segmentation Notebook
This notebook will walkthrough all the steps for performing YOLOv8 object detections and segmentations on images while in Google Colab. We will be using yolov8 for this tutorial.

***00.INITIAL SETUP***

00.01: Install Required Library, Ultralytics Library

In [ ]:
from IPython.display import Image

In [ ]:
!pip install ultralytics==8.0.0

00.02: Check whether or not GPU is available

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

IF the above cell says **'True'** after running > proceed to next cell

IF the above cell says **'False'** make sure Runtime>Change Runtime Type>Hardware Accellerator is set to GPU

In [ ]:
torch.__version__

***01.TAKE WEBCAM PICTURE***

01.01: Webcam Setup

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

01.02: Take Webcam Image (it will be saved as "photo.jpg")

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

***02.WEBCAM IMAGE OBJECT DETECTION*** 

02.01: Run detection on image

In [ ]:
!yolo task=detect mode=predict model=yolov8s.pt source='/content/photo.jpg' 

02.02: Display Output Image

In [ ]:
Image('/content/runs/detect/predict/photo.jpg')

*NOTE - if you have tested multiple images in a row the folder name will increase each time, and you will need to edit the above code to match the folder you want to display*

***03.WEBCAM IMAGE SEGMENTATION*** 

03.01: Run segmentation on image

In [ ]:
!yolo task=segment mode=predict model=yolov8s-seg.pt source='/content/photo.jpg' 

03.02: Display output image

In [ ]:
Image('/content/runs/segment/predict/photo.jpg')

*NOTE - if you have tested multiple images in a row the folder name will increase each time, and you will need to edit the above code to match the folder you want to display*

_________________________________________________________________

***04.ALTERNATIVE OPTION - UPLOAD IMAGE***



04.01: Run next cell to upload an image 
> - must be smaller than SD or you run out of memory
> - you can upload multiples but you can only detect or segment one at a time

In [ ]:
from google.colab import files
uploaded = files.upload()

04.02: Choose the image you want to test first, find it in the file structure to the left, hover over it, click the dots and copy the path into the cell below

In [ ]:
#@markdown **UPLOADED IMAGE PATH**
image_path = "/content/maskImage.png" #@param {type:"string"}
#image = str(image_path) - str(image_path[9:])
image = image_path.replace('/content/', '')

***05.UPLOADED IMAGE OBJECT DETECTION*** 

05.01: Run detection on image

In [ ]:
# Importing the modules
import os
import shutil

shutil.copy(image_path,'/content/tmpimg.jpg') #copy the og file to temporary folder

%cd /content/

!yolo task=detect mode=predict model=yolov8s.pt source='/content/tmpimg.jpg' 

05.02: Display Output Image

In [ ]:
Image('/content/runs/detect/predict/tmpimg.jpg')

*NOTE - if you have tested multiple images in a row the folder name will increase each time, and you will need to edit the above code to match the folder you want to display*

***06.UPLOADED IMAGE SEGMENTATION*** 

06.01: Run Segmentation on Image

In [ ]:
!yolo task=segment mode=predict model=yolov8s-seg.pt source='/content/tmpimg.jpg' 

06.02: Display output image

In [ ]:
Image('/content/runs/segment/predict/tmpimg.jpg')

*NOTE - if you have tested multiple images in a row the folder name will increase each time, and you will need to edit the above code to match the folder you want to display*

________________________________________________________________


This Colab Notebook was created by Laure Michelon for her course. The notebook builds off of [THIS NOTEBOOK](https://colab.research.google.com/drive/1Chb8DSVcmmsOmse312NLitRokYsR6-hi?usp=sharing) with tweaks made for class. 